# 1、RNNCell方法

In [ ]:
import torch

input_size = 4
hidden_size = 4
batch_size = 1


In [ ]:
idx2char = ['e','h','l','o']

x_data = [1,0,2,2,3]
y_data = [3,1,2,3,2]

one_hot_lookup = [[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1]]
# one hot 编码
x_one_hot = [one_hot_lookup[x] for x in x_data]
# inputs: (seq_len,batch_size,input_size) (5,1,4)
inputs = torch.Tensor(x_one_hot).view(-1,batch_size,input_size)
# 此时 print(labels[0].shape) 为空，但我们需要他有一个维度。
# labels = torch.LongTensor(y_data).view(-1)

labels = torch.LongTensor(y_data).view(-1,1)

# print(labels[0].shape)

In [3]:
class Model(torch.nn.Module):
    def __init__(self,input_size,hidden_size,batch_size):
        super(Model,self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.rnncell = torch.nn.RNNCell(self.input_size,self.hidden_size)

    def forward(self,input,hidden):
        hidden = self.rnncell(input,hidden)
        return hidden
    
    def init_hidden(self):
        return torch.zeros(self.batch_size,self.hidden_size)
    
net = Model(input_size,hidden_size,batch_size)

In [4]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.1)

In [ ]:
for epoch in range(15):
    loss = 0
    optimizer.zero_grad()
    hidden = net.init_hidden()
    print('Predicted string:',end=' ')
    flag = 0
    # 模型进行一次完整的前向传播
    for input,label in zip(inputs,labels):
        # 前向传播
        hidden = net(input,hidden)
        if flag == 0:
            print(f'input:{input},label:{label},hidden:{hidden}')
            flag = 1
        # 计算损失
        loss += criterion(hidden,label)
        # 得到h_t的对应字符
        _,idx = hidden.max(dim=1)
        print(idx2char[idx.item()],end='')
    # 反向传播
    loss.backward()
    # 更新参数
    optimizer.step()
    print(', Epoch [%d/15] loss=%.4f' % (epoch+1,loss.item()))


NameError: name 'optimizer' is not defined

# 2、RNN方法

In [4]:
import torch

input_size = 4
hidden_size = 4
num_layers = 1
batch_size = 1
seq_len = 5

In [ ]:
idx2char = ['e','h','l','o']

x_data = [1,0,2,2,3]
y_data = [3,1,2,3,2]

one_hot_lookup = [[1,0,0,0],
                  [0,1,0,0],
                  [0,0,1,0],
                  [0,0,0,1]]
x_one_hot = [one_hot_lookup[x] for x in x_data]

inputs = torch.Tensor(x_one_hot).view(seq_len,batch_size,input_size)
# shape: (seq_len, batch_size, input_size)

# torch.LongTensor 是 PyTorch 中的一个数据类型，用于表示包含整数（整型数据）的张量（tensor）。
# 它是一种特定的张量类型，其中的元素都为整数类型，并且使用 64 位整数进行存储。
labels = torch.LongTensor(y_data)
# 推荐使用torch.tensor()
labels = torch.tensor(y_data,dtype=torch.long)
# shape: (seq_len * batch_size, 1)

# print(labels[0].shape)




torch.return_types.max(
values=tensor([[1., 1., 1., 1.]]),
indices=tensor([[1, 0, 2, 4]]))

In [ ]:
class RNN_Model(torch.nn.Module):
    def __init__(self,input_size,hidden_size,batch_size,num_layers):
        super(RNN_Model,self).__init__()
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.batch_size = batch_size
        self.rnn = torch.nn.RNN(self.input_size,self.hidden_size,num_layers=self.num_layers)


    def forward(self,input):
        # input shape: (seq_len, batch_size, input_size)
        # hidden0 shape: (num_layers, batch_size, hidden_size)
        hidden0 = torch.zeros(self.num_layers,self.batch_size,self.hidden_size)
        # _是RNN的隐藏状态hidden，我们不需要它
        out,_ = self.rnn(input,hidden0)
        # out shape: (seq_len *batch_size, hidden_size)
        return out.view(-1,self.hidden_size)
    
net = RNN_Model(input_size,hidden_size,batch_size,num_layers)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(),lr=0.1)

for epoch in range(15):
    optimizer.zero_grad()
    outputs = net(inputs)
    # inputs shape: (seq_len, batch_size, input_size)
    # outputs shape: (seq_len * batch_size, hidden_size)
    # labels shape: (seq_len * batch_size, 1)
    loss = criterion(outputs,labels)
    # 反向传播
    loss.backward()
    # 更新参数
    optimizer.step()

    # 得到h_t的预测字母结果对应的索引
    # 第一个参数是dim=1的所有最大值，第二个参数是维度的索引
    _,idx = outputs.max(dim=1)
    # 将索引转化为numpy数组
    idx = idx.data.numpy()
    # 迭代idx数组，将每个索引对应的字母打印出来
    print('Predicted: ',''.join([idx2char[x] for x in idx]),end='')
    # .item() 将张量中的单个值转换为 Python 数字
    # .data.numpy() 将张量转换为 NumPy 数组
    print(', Epoch [%d/15] loss=%.4f' % (epoch+1,loss.item()))

out.shape: torch.Size([5, 1, 4])
out.shape: torch.Size([5, 1, 4])
Predicted:  oehll, Epoch [1/15] loss=1.3548
out.shape: torch.Size([5, 1, 4])
out.shape: torch.Size([5, 1, 4])
Predicted:  ollll, Epoch [2/15] loss=1.1208
out.shape: torch.Size([5, 1, 4])
out.shape: torch.Size([5, 1, 4])
Predicted:  ollol, Epoch [3/15] loss=1.0043
out.shape: torch.Size([5, 1, 4])
out.shape: torch.Size([5, 1, 4])
Predicted:  ooooo, Epoch [4/15] loss=0.9150
out.shape: torch.Size([5, 1, 4])
out.shape: torch.Size([5, 1, 4])
Predicted:  ohooo, Epoch [5/15] loss=0.8360
out.shape: torch.Size([5, 1, 4])
out.shape: torch.Size([5, 1, 4])
Predicted:  ohooo, Epoch [6/15] loss=0.7734
out.shape: torch.Size([5, 1, 4])
out.shape: torch.Size([5, 1, 4])
Predicted:  ohool, Epoch [7/15] loss=0.7200
out.shape: torch.Size([5, 1, 4])
out.shape: torch.Size([5, 1, 4])
Predicted:  ohool, Epoch [8/15] loss=0.6727
out.shape: torch.Size([5, 1, 4])
out.shape: torch.Size([5, 1, 4])
Predicted:  ohool, Epoch [9/15] loss=0.6270
out.shape: